In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

In [ ]:
!apt install enchant
!pip install pyenchant
!pip3 install pyenchant==1.6.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import nltk
import enchant
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from gensim import models
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
def tokenizarTexto(x):
  d = enchant.Dict("en")
  x=x.casefold()
  x=re.sub(r'http\S*', '',x)
  x=re.sub(r'[^a-z\s]', '',x)
  lemma=WordNetLemmatizer()
  x=lemma.lemmatize(x)
  tokens=nltk.word_tokenize(x)
  #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
  #tokens = tknzr.tokenize(x)
  l=stopwords.words('english')+list(punctuation)
  tokens = [word for word in tokens if ((word not in l) & (d.check(word)))]
  return tokens

In [ ]:
import re
def docToVec (train,datos):
  l=[]
  datosTweets=datos.apply(lambda x:[tokenizarTexto(x['text']),x['id']],axis=1).tolist()
  trainTweets=train.apply(lambda x:[tokenizarTexto(x['text']),x['id']],axis=1).tolist()
  for vectores in datosTweets:
    l.append(TaggedDocument(words=vectores[0], tags=str(vectores[1])))
  for vectores in trainTweets:
    l.append(TaggedDocument(words=vectores[0], tags=str(vectores[1])))
  max_epochs = 300
  vec_size = 200
  model = Doc2Vec(vector_size=vec_size, epochs=max_epochs)  
  model.build_vocab(l)
  model.train(l,total_examples=model.corpus_count,epochs=max_epochs)
  model.save('do2vec.model')
  ldatos=[]
  ltrain=[]
  for vectores in datosTweets:
    ldatos.append(model.infer_vector(vectores[0]))
  dfdatos= pd.DataFrame(ldatos)
  for vectores in trainTweets:
    ltrain.append(model.infer_vector(vectores[0]))
  dftrain= pd.DataFrame(ltrain)
  return (dfdatos,dftrain)

(dfdatos,dftrain) = docToVec(train,datos)
dfdatos.to_csv('dfdatos.csv',encoding='utf-8',index=False)
dftrain.to_csv('dftrain.csv',encoding='utf-8',index=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
!ls

catboost_info	       dftrain	       sample_submission.csv  XGBoost.ipynb
CatBoost.ipynb	       do2vec.model    test.csv
colsample_bylevel.png  kaggle.json     train.csv
dfdatos		       resultados.csv  Untitled0.ipynb
